In [14]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [15]:
df_map = pd.read_csv("../dataset/created/df_map.csv")

In [16]:
fig = px.scatter_map(
    df_map,
    lat="ORIGIN_LAT",
    lon="ORIGIN_LON",
    size="DELAY_OVERALL",
    color="DELAY_OVERALL",
    hover_name="ORIGIN",
    hover_data={"CANCELLED": True, "DIVERTED": True, "DELAY_OVERALL": True},
    color_continuous_scale="Reds",
    zoom=3,
    height=600
)

fig.update_layout(
    map_style="open-street-map",
    title="Average Flight Delays by Origin Airport - top 50 more delay overall"
)

fig.show()

In [17]:
def criar_mapa_rotas_atraso(df, top_n=50, altura=800):
    """
    Cria um mapa interativo das rotas de voo coloridas por média de atraso
    
    Parameters:
    df: DataFrame com dados dos voos
    top_n: número de rotas principais para mostrar
    altura: altura do mapa em pixels
    """
    
    # --- PRÉ-PROCESSAMENTO DOS DADOS ---
    # Calcular atraso médio por rota
    rotas_atraso = (
        df.groupby(["ORIGIN", "DEST", "ORIGIN_LAT", "ORIGIN_LON", "DEST_LAT", "DEST_LON"], 
                  as_index=False)["DELAY_OVERALL"]
        .mean()
        .sort_values(by="DELAY_OVERALL", ascending=False)
        .head(top_n)
        .dropna(subset=['ORIGIN_LAT', 'ORIGIN_LON', 'DEST_LAT', 'DEST_LON'])
    )
    
    # Calcular estatísticas
    min_atraso = rotas_atraso['DELAY_OVERALL'].min()
    max_atraso = rotas_atraso['DELAY_OVERALL'].max()
    
    print(f"📊 Estatísticas das {len(rotas_atraso)} rotas:")
    print(f"   • Atraso mínimo: {min_atraso:.1f} min")
    print(f"   • Atraso máximo: {max_atraso:.1f} min")
    print(f"   • Atraso médio: {rotas_atraso['DELAY_OVERALL'].mean():.1f} min")
    
    # --- CRIAÇÃO DO MAPA PRINCIPAL ---
    fig = go.Figure()
    
    # Adicionar cada rota ao mapa
    for idx, rota in rotas_atraso.iterrows():
        # Normalizar o atraso para cores (0-1)
        intensidade_cor = (rota['DELAY_OVERALL'] - min_atraso) / (max_atraso - min_atraso) if max_atraso > min_atraso else 0.5
        
        # Escala de cores: verde (baixo) → amarelo → laranja → vermelho (alto)
        cor = px.colors.sample_colorscale("RdYlGn_r", [intensidade_cor])[0]
        
        # Largura da linha proporcional ao atraso
        largura_linha = 1 + (intensidade_cor * 4)
        
        fig.add_trace(go.Scattergeo(
            lon=[rota['ORIGIN_LON'], rota['DEST_LON']],
            lat=[rota['ORIGIN_LAT'], rota['DEST_LAT']],
            mode='lines+markers',
            line=dict(
                width=largura_linha,
                color=cor,
            ),
            marker=dict(
                size=[8, 10],  # Origem menor, destino maior
                color=cor,
                symbol='circle',
                line=dict(width=1, color='white')
            ),
            name=f"{rota['ORIGIN']} → {rota['DEST']}",
            text=f"{rota['ORIGIN']} → {rota['DEST']}",
            customdata=[[rota['DELAY_OVERALL']]],
            hovertemplate=(
                "<b>%{text}</b><br>"
                "Atraso Médio: <b>%{customdata[0]:.1f} minutos</b><br>"
                "Origem: %{lat[0]:.2f}°N, %{lon[0]:.2f}°W<br>"
                "Destino: %{lat[1]:.2f}°N, %{lon[1]:.2f}°W<br>"
                "<extra></extra>"
            ),
            showlegend=False
        ))
    
    # --- BARRA DE CORES ---
    fig.add_trace(go.Scattergeo(
        lon=[None], lat=[None],
        mode='markers',
        marker=dict(
            size=0,
            colorscale="RdYlGn_r",  # Invertido: vermelho (alto) → verde (baixo)
            cmin=min_atraso,
            cmax=max_atraso,
            colorbar=dict(
                title=dict(
                    text="<b>Atraso Médio (minutos)</b>",
                    font=dict(size=12)
                ),
                thickness=15,
                len=0.6,
                x=1.02,
                y=0.5,
                tickfont=dict(size=10)
            ),
            showscale=True
        ),
        hoverinfo='none',
        showlegend=False
    ))
    
    # --- LAYOUT DO MAPA ---
    fig.update_layout(
        title=dict(
            text=f"<b>Top {top_n} Rotas com Maiores Atrasos Médios</b><br>"
                 f"<sup>Cor e espessura indicam intensidade do atraso</sup>",
            x=0.5,
            xanchor='center',
            font=dict(size=20, color='#2c3e50')
        ),
        geo=dict(
            scope='north america',
            projection_type='albers usa',
            showland=True,
            landcolor='rgb(240, 240, 240)',
            countrycolor='rgb(200, 200, 200)',
            coastlinecolor='rgb(160, 160, 160)',
            lakecolor='rgb(85, 173, 240)',
            showsubunits=True,
            subunitcolor='rgb(220, 220, 220)',
            showlakes=True,
            showcountries=True,
            fitbounds="locations"
        ),
        height=altura,
        margin=dict(l=0, r=0, t=80, b=0),
        paper_bgcolor='white',
        plot_bgcolor='white',
        hovermode='closest'
    )
    
    return fig

# --- VERSÃO ALTERNATIVA: MAPA DE AEROPORTOS ---
def criar_mapa_aeroportos_atraso(df, altura=700):
    """
    Cria mapa de aeroportos coloridos por atraso médio
    """
    
    # Calcular atraso médio por aeroporto de origem
    atrasos_aeroporto = (
        df.groupby(["ORIGIN", "ORIGIN_LAT", "ORIGIN_LON"], as_index=False)
        .agg({
            'DELAY_OVERALL': 'mean',
            'CANCELLED': 'sum',
            'DIVERTED': 'sum',
            'FL_DATE': 'count'  # Número de voos
        })
        .rename(columns={'FL_DATE': 'TOTAL_VOOS'})
        .sort_values('DELAY_OVERALL', ascending=False)
    )
    
    fig = px.scatter_mapbox(
        atrasos_aeroporto,
        lat="ORIGIN_LAT",
        lon="ORIGIN_LON",
        size="TOTAL_VOOS",
        color="DELAY_OVERALL",
        hover_name="ORIGIN",
        hover_data={
            "DELAY_OVERALL": ":.1f",
            "TOTAL_VOOS": True,
            "CANCELLED": True,
            "DIVERTED": True,
            "ORIGIN_LAT": False,
            "ORIGIN_LON": False
        },
        color_continuous_scale="RdYlGn_r",
        size_max=15,
        zoom=3,
        height=altura,
        title="Atrasos Médios por Aeroporto de Origem"
    )
    
    fig.update_layout(
        mapbox_style="carto-positron",
        margin=dict(l=0, r=0, t=40, b=0),
        coloraxis_colorbar=dict(
            title="Atraso Médio (min)",
            thickness=15
        )
    )
    
    return fig

# --- EXECUÇÃO ---
if __name__ == "__main__":
    # Criar mapa de rotas (sua versão principal)
    fig_rotas = criar_mapa_rotas_atraso(df_map, top_n=50, altura=800)
    fig_rotas.show()
    
    # Criar mapa de aeroportos (versão alternativa)
    fig_aeroportos = criar_mapa_aeroportos_atraso(df_map, altura=600)
    fig_aeroportos.show()

📊 Estatísticas das 50 rotas:
   • Atraso mínimo: 1363.0 min
   • Atraso máximo: 2687.0 min
   • Atraso médio: 1628.5 min


C:\Users\Vitoria\AppData\Local\Temp\ipykernel_41708\2024486761.py:149: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

